In [ ]:
import pandas as pd
dates= pd.date_range('1/1/2018', '8/1/2018')

In [ ]:
for item in dates:
#     http://alihz-net-0.qbtrade.org/hist-ticks?date=2018-01-02&contract=okex/btc.usdt
    a = str(item).split(' ')[0]
    print "curl -o " + a + ".json \"http://alihz-net-0.qbtrade.org/hist-ticks?date="+ a + "&contract=okex/btc.usdt\""
#     print "curl -o " + a + ".json \"https://1token.trade/api/v1/quote/zhubi?contract=binance/btc.usdt&since="+a+"T00:00:00Z&until="+a+"T23:59:59Z&size=2000\""

In [ ]:
#coding=utf-8
import sunburnt
import urllib
from pymongo import *
from bson.objectid import ObjectId
import logging
from datetime import datetime
import json
from time import mktime
from feedparser import _parse_date as parse_date
import time
import sys
import getopt
import ConfigParser
args = sys.argv[1:]
optlist, args = getopt.getopt(args, 'c:')
cmd_opt = {}
print args,optlist,cmd_opt

In [ ]:
for opt in optlist:
  cmd_opt[opt[0]] = opt[1]
conf_file = cmd_opt['-c']
config = ConfigParser.ConfigParser()
config.read(conf_file)
hostname = config.get("mongodb", "hostname")
port_num = int(config.get("mongodb", "port_num"))
db_name = config.get("mongodb", "db")
connection = Connection(hostname, port_num)
db = connection[db_name]
courseTable = db.course
lecTable = db.lecture

In [ ]:
try:
  f = file("json1-14/14.json")
  s = json.load(f)
  courseData = s["results"]["course"]
  lecDataArr = s["results"]["lecture"]
  f.close
  print "get file content successfully!"
  #insert course
  courseId = courseTable.save(courseData)
  courseId = str(courseId)
  print "courseId: "+courseId
  print "lec length: "+str(len(lecDataArr))
  #insert lecture
  lecIdArr = []
  for lecData in lecDataArr:
    lecData["course_id"] = courseId
    lecId = lecTable.save(lecData)
    lecIdArr.append(str(lecId))
  # update course
  courseTable.update({'_id':ObjectId(courseId)},
            {"$set":{"lectures.lecture_id_list":lecIdArr}},
            upsert=True, multi=True);
  print 'insert successfully!'
except Exception, e:
  print e

In [ ]:
#####################################################################################################################################
from pymongo import MongoClient
from bson.objectid import ObjectId
from read_json  import read



def mongoclient(data):
    client = MongoClient('127.0.0.1', 27017)
    db = client.test
    collection = db.test
    # collection.insert({})
    # for i in collection.find({"name":"乐清市东岸北段"}):
    #      print(i)
#     data = ('coastlines.real.json')
    find_Da = ('VnBitCoin.bitcoinTICK')
    find_Id = ('')
    find_Node = ('')

    want_Data = read(data, find_Da)  #传入json文件，根据关键字查询
    want_Id = read(data, find_Id)
    want_No = read(data, find_Node)
    id = {'_id': ObjectId(want_Id)}
    id_print = collection.find_one(id)
    print(id_print)

    collection.update((id), {'$set': {'track': want_Data}})   #根据ObjectID修改track
    collection.update((id), {'$push': {'nodeindex': want_No}})  #根据ObjectID添加nodeindex
    collection.update((id), {'$set': {'nodeindex': want_No}})   #根据ObjectID修改nodeindex ,去除多余的[]


    id_change = collection.find_one(id)   #调出修改后的数据
    print("修改后:")
    print(id_change)




if __name__ =='__main__':
    mongoclient()

<---------------读取文件代码----------------------->
import json

def read(data, find):
    # 设置以utf-8解码模式读取文件，encoding参数必须设置，否则默认以gbk模式读取文件，当文件中包含中文时，会报错
    f = open(data, encoding='utf-8')
    setting = json.load(f)

    # 注意多重结构的读取语法
    # family = setting['BaseSettings']['font']
    style = setting[find]


    # print(family)
    return style

In [ ]:
#####################################################################################################################################
from mongodbHandler import MongoDbHandler 
from pandas import DataFrame 
finallyResult = {} 
finallyResult["aa"]="bb" 
finallyResult["cc"]="dd" 
df = DataFrame(finallyResult,index=[1]) 
data = df.to_dict('records') 
mongoSession = MongoDbHandler('localhost')#A为mongodb的name,B为用户名,C为密码 
mongoSession.insert_many("VnBitCoin", "bitcoinTICK", data)#D为数据库名,E为集合名 
time.sleep(2) 
mongoSession.close() 

In [3]:

#!/usr/bin/env python
# -*- coding:utf-8 -*-

from pymongo import MongoClient
conn = MongoClient('127.0.0.1', 27017)
#db = conn.VnBitCoin  #连接mydb数据库，没有则自动创建
#my_set = db.bitcoinTICK#使用test_set集合，没有则自动创建
db = conn.VnTrader_Tick_Db  #连接mydb数据库，没有则自动创建
my_set = db.bitcoin#使用test_set集合，没有则自动创建

In [7]:
import os,json
from pymongo.mongo_client import MongoClient
import datetime
rootdir = "E:\\1tokenDownload"
tmplist = []
for root,dirs,paths in os.walk(rootdir):
    for pathfile in paths:
        filepath = (os.path.join(root,pathfile))
        #你想对文件的操作
        with open(filepath) as f:
            for line in f:
                line = line.replace('+08:00','Z')
                dayline = json.loads(line)
                dayline['datetime']=datetime.datetime.strptime(str(dayline['time']),'%Y-%m-%dT%H:%M:%S.%fZ')
                for i in range(1,6):
                    dayline['askPrice%d'%i]= dayline['asks'][i-1]['price']
                    dayline['askVolume%d'%i]= dayline['asks'][i-1]['volume']
                    dayline['bidPrice%d'%i]= dayline['bids'][i-1]['price']
                    dayline['bidVolume%d'%i]= dayline['bids'][i-1]['volume']
                dayline['upperLimit']=100000
                dayline['lowerLimit']=0
                dayline['openPrice']=''
                dayline['rawData']=''
                dayline['highPrice']=''
                dayline['preClosePrice']=''
                dayline['gatewayName']=''
                dayline['exchange']=''
                dayline['symbol']=''
                time = dayline['time'].replace('Z','')
                dayline['date']=time.split('T')[0]
                dayline['time']=time.split('T')[1]
                dayline['vtSymbol']='bitcoin'
                del dayline['asks']
                del dayline['bids']
                my_set.insert_one(dayline) 
#                 print dayline
#                 break
#         break
#     break    
#                 my_set.insert_one(dayline) 
#                 tmplist.append(dayline)
#         print len(tmplist)
#         my_set.insert_many(tmplist) 
        
# list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
# for i in range(0,len(list)):

#     path = os.path.join(rootdir,list[i])
#     if os.path.isfile(path):

KeyboardInterrupt: 

In [ ]:
del tmplist[0]['asks']

In [ ]:
tmplist